In [ ]:
#data augmentation for -1 and 1 labels

from collections import Counter
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

# Adjust the path to the file location in your Google Drive
data_path = 'chess_images/prepared_data/hog_svm_data.npy'

# Load the prepared data
data = np.load(data_path, allow_pickle=True).item()


# Extract features and labels
X = data["features"]
y = data["labels"]


def augment_features(X, y, target_label, num_samples):
    augmented_X, augmented_y = [], []
    for i, (features, label) in enumerate(zip(X, y)):
        if label == target_label:
            for _ in range(num_samples):
                # Add small noise to features
                noisy_features = features + np.random.normal(0, 0.01, size=features.shape)
                augmented_X.append(noisy_features)
                augmented_y.append(label)
    return np.vstack([X, np.array(augmented_X)]), np.hstack([y, np.array(augmented_y)])

# Augment samples for label -1 and 1
X_augmented, y_augmented = augment_features(X, y, target_label=-1, num_samples=2)
X_augmented, y_augmented = augment_features(X_augmented, y_augmented, target_label=1, num_samples=2)
X_augmented, y_augmented = augment_features(X_augmented, y_augmented, target_label=0, num_samples=2)

# Check the new label distribution
print(Counter(y_augmented))
print(X_augmented.shape)



MemoryError: Unable to allocate 1.85 GiB for an array with shape (192000, 1296) and data type float64

In [19]:
#K Fold and Save Model
import numpy as np
import joblib
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Load dataset
# data = np.load("chess_images/prepared_data/hog_svm_data.npy", allow_pickle=True).item()
# X, y = data["features"], data["labels"]

# Define models
models = {
    "SVM": SVC(kernel="linear"),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    # "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
}

# Use Stratified K-Fold (better for classification tasks)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation for each model and save the best one
best_model = None
best_score = 0

for name, model in models.items():
    scores = cross_val_score(model,X_augmented, y_augmented, cv=kfold, scoring="accuracy")
    mean_score = scores.mean()
    print(f"{name}: Mean Accuracy = {mean_score:.4f}, Std Dev = {scores.std():.4f}")

    # Train model on the full dataset before saving
    model.fit(X_augmented, y_augmented)
    model_path = f"chess_images/prepared_data/{name.lower().replace(' ', '_')}_model.pkl"
    joblib.dump(model, model_path)
    print(f"Saved {name} model to {model_path}")

    # Track the best model
    # if mean_score > best_score:
    #     best_score = mean_score
    #     best_model = model

# Save the best model separately
# best_model_path = "chess_images/prepared_data/best_model.pkl"
# joblib.dump(best_model, best_model_path)
# print(f"Best model saved to {best_model_path}")

SVM: Mean Accuracy = 0.9987, Std Dev = 0.0008
Saved SVM model to chess_images/prepared_data/svm_model.pkl
Random Forest: Mean Accuracy = 0.9997, Std Dev = 0.0006
Saved Random Forest model to chess_images/prepared_data/random_forest_model.pkl


In [ ]:
import numpy as np
import joblib
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
# from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Load dataset
# data = np.load("chess_images/prepared_data/hog_svm_data.npy", allow_pickle=True).item()
# X, y = data["features"], data["labels"]

# Define models
models = {
    # "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
    "LightGBM": LGBMClassifier(),
    # "CatBoost": CatBoostClassifier(verbose=0),
    "Extra Trees": ExtraTreesClassifier(n_estimators=100),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100)
}

# Use Stratified K-Fold (better for classification tasks)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation for each model and save the best one
best_model = None
best_score = 0

for name, model in models.items():
    scores = cross_val_score(model, X_augmented, y_augmented, cv=kfold, scoring="accuracy")
    mean_score = scores.mean()
    print(f"{name}: Mean Accuracy = {mean_score:.4f}, Std Dev = {scores.std():.4f}")

    # Train model on the full dataset before saving
    model.fit(X_augmented, y_augmented)
    model_path = f"chess_images/prepared_data/{name.lower().replace(' ', '_')}_model.pkl"
    joblib.dump(model, model_path)
    print(f"Saved {name} model to {model_path}")

    # Track the best model
    if mean_score > best_score:
        best_score = mean_score
        best_model = model

# Save the best model separately
best_model_path = "chess_images/prepared_data/best_model.pkl"
joblib.dump(best_model, best_model_path)
print(f"Best model saved to {best_model_path}")


In [5]:
#loading the model and testing input as 480*480 image and output as bw_fen file

import joblib
import cv2
import numpy as np
from skimage.feature import hog

# Load the trained model
model = joblib.load('chess_images/prepared_data/svm_model.pkl')

def extract_hog_features(image):
    image = cv2.resize(image, (60, 60))
    fd, _ = hog(image, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True)
    return fd

def split_and_predict_fen(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (480, 480))
    step = 60
    label_mapping_inverse = {1: "w", -1: "b", 0: "1"}  # FEN compatible labels
    predictions = []

    for row in range(8):
        row_data = []
        for col in range(8):
            square = image[row * step:(row + 1) * step, col * step:(col + 1) * step]
            features = extract_hog_features(square)
            prediction = model.predict([features])[0]
            row_data.append(label_mapping_inverse[prediction])
        # Convert row data to FEN row format
        fen_row = ''.join(row_data)
        # Consolidate empty squares into numbers
        compact_fen_row = ''
        count = 0
        for char in fen_row:
            if char == '1':
                count += 1
            else:
                if count > 0:
                    compact_fen_row += str(count)
                    count = 0
                compact_fen_row += char
        if count > 0:
            compact_fen_row += str(count)
        predictions.append(compact_fen_row)

    # Join rows with '/' for the final FEN
    fen_result = '/'.join(predictions)
    return fen_result

# Example usage:
image_path = 'chess_images/test1.jpg'
fen_output = split_and_predict_fen(image_path)
print("Generated FEN:", fen_output)

# Optionally, save the FEN to a file
with open("generated_bw_fen.fen", "w") as file:
    file.write(fen_output)


Generated FEN: 2bwbw2/ww4wb/wwbwww1w/3bbb2/3wbb2/b6b/bbbbbbbb/8


In [47]:
import joblib
import cv2
import numpy as np
from skimage.feature import hog

# Load the trained model
model = joblib.load('chess_images/prepared_data/hog_random_forest_model.pkl')

def extract_hog_features(image):
    image = cv2.resize(image, (60, 60))
    fd, _ = hog(image, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True)
    return fd

def split_and_predict_fen_with_overlay(image_path, output_path='overlay_result.jpg'):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    color_image = cv2.imread(image_path)  # Read color version for overlay
    image = cv2.resize(image, (480, 480))
    color_image = cv2.resize(color_image, (480, 480))

    step = 60
    label_mapping_inverse = {1: "w", -1: "b", 0: "1"}  # FEN compatible labels

    predictions = []

    for row in range(8):
        row_data = []
        for col in range(8):
            square = image[row * step:(row + 1) * step, col * step:(col + 1) * step]
            features = extract_hog_features(square)
            prediction = model.predict([features])[0]
            label = label_mapping_inverse[prediction]

            # Add to FEN processing
            row_data.append(label)

            # Draw on image overlay if there is a piece
            if label in ['w', 'b']:
                text_color = (255, 255, 255) if label == 'w' else (0, 0, 0)
                cv2.putText(color_image, label, (col * step + 20, row * step + 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)

        # Convert row data to compact FEN row
        fen_row = ''.join(row_data)
        compact_fen_row = ''
        count = 0
        for char in fen_row:
            if char == '1':
                count += 1
            else:
                if count > 0:
                    compact_fen_row += str(count)
                    count = 0
                compact_fen_row += char
        if count > 0:
            compact_fen_row += str(count)

        predictions.append(compact_fen_row)

    # Final FEN string
    fen_result = '/'.join(predictions)
    print("Generated FEN:", fen_result)

    # Save the image with overlay
    cv2.imwrite(output_path, color_image)

    # Optionally, save the FEN to a file
    # with open("generated_bw_fen.fen", "w") as file:
    #     file.write(fen_result)

# Example usage:
image_path = 'chess_images/test.jpg'
split_and_predict_fen_with_overlay(image_path, output_path='chess_images/overlay_result.jpg')


Generated FEN: 2wwww2/ww4ww/wwwwwwww/3www2/3www2/w6w/wwwwwwww/8


In [ ]:
# Occupy or Empty Test

import joblib
import cv2
import numpy as np
from skimage.feature import hog

# Load the trained model
model = joblib.load('chess_images/prepared_data/tf_hog_random_forest_model.pkl')

def extract_hog_features(image):
    image = cv2.resize(image, (60, 60))
    fd, _ = hog(image, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True)
    return fd

def split_and_predict_fen_with_overlay(image_path, output_path='overlay_result.jpg'):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    color_image = cv2.imread(image_path)  # Read color version for overlay
    image = cv2.resize(image, (480, 480))
    color_image = cv2.resize(color_image, (480, 480))

    step = 60
    label_mapping_inverse = {1: "O", 0: "1"}  # FEN compatible labels

    predictions = []

    for row in range(8):
        row_data = []
        for col in range(8):
            square = image[row * step:(row + 1) * step, col * step:(col + 1) * step]
            features = extract_hog_features(square)
            prediction = model.predict([features])[0]
            label = label_mapping_inverse[prediction]

            # Add to FEN processing
            row_data.append(label)

            # Draw on image overlay if there is a piece
            if label in ['O', 'b']:
                text_color = (255, 255, 255) if label == 'w' else (0, 0, 0)
                cv2.putText(color_image, label, (col * step + 20, row * step + 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)

        # Convert row data to compact FEN row
        fen_row = ''.join(row_data)
        compact_fen_row = ''
        count = 0
        for char in fen_row:
            if char == '1':
                count += 1
            else:
                if count > 0:
                    compact_fen_row += str(count)
                    count = 0
                compact_fen_row += char
        if count > 0:
            compact_fen_row += str(count)

        predictions.append(compact_fen_row)

    # Final FEN string
    fen_result = '/'.join(predictions)
    print("Generated FEN:", fen_result)

    # Save the image with overlay
    cv2.imwrite(output_path, color_image)

# Example usage:
image_path = 'chess_images/test1.jpg'
split_and_predict_fen_with_overlay(image_path, output_path='chess_images/overlay_result.jpg')


Generated FEN: 2OOOO2/OO4OO/OOOOOOOO/3OOO2/3OOO2/O6O/OOOOOOOO/8


In [2]:
import joblib
import cv2
import numpy as np

# Load the trained model
model = joblib.load('chess_images/prepared_data/bw_hsv_random_forest_model.pkl')

def extract_hsv_features(image, bins=16):
    """Extracts HSV color histograms from an image."""
    # Convert to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Compute histograms for each channel (H, S, V)
    h_hist = cv2.calcHist([hsv_image], [0], None, [bins], [0, 180])  # Hue
    s_hist = cv2.calcHist([hsv_image], [1], None, [bins], [0, 256])  # Saturation
    v_hist = cv2.calcHist([hsv_image], [2], None, [bins], [0, 256])  # Value

    # Normalize histograms
    h_hist = cv2.normalize(h_hist, h_hist).flatten()
    s_hist = cv2.normalize(s_hist, s_hist).flatten()
    v_hist = cv2.normalize(v_hist, v_hist).flatten()

    # Concatenate histograms into a feature vector
    return np.concatenate((h_hist, s_hist, v_hist))

def split_and_predict_fen_with_overlay(image_path, output_path='overlay_result.jpg'):
    image = cv2.imread(image_path)
    color_image = image.copy()  # Keep original for overlay
    image = cv2.resize(image, (480, 480))
    color_image = cv2.resize(color_image, (480, 480))

    step = 60
    label_mapping_inverse = {1: "w", -1: "b", 0: "1"}  # FEN compatible labels

    predictions = []

    for row in range(8):
        row_data = []
        for col in range(8):
            square = image[row * step:(row + 1) * step, col * step:(col + 1) * step]
            features = extract_hsv_features(square)
            prediction = model.predict([features])[0]
            label = label_mapping_inverse[prediction]

            # Add to FEN processing
            row_data.append(label)

            # Draw on image overlay if there is a piece
            if label in ['w', 'b']:
                text_color = (255, 255, 255) if label == 'w' else (0, 0, 0)
                cv2.putText(color_image, label, (col * step + 20, row * step + 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)

        # Convert row data to compact FEN row
        fen_row = ''.join(row_data)
        compact_fen_row = ''
        count = 0
        for char in fen_row:
            if char == '1':
                count += 1
            else:
                if count > 0:
                    compact_fen_row += str(count)
                    count = 0
                compact_fen_row += char
        if count > 0:
            compact_fen_row += str(count)

        predictions.append(compact_fen_row)

    # Final FEN string
    fen_result = '/'.join(predictions)
    print("Generated FEN:", fen_result)

    # Save the image with overlay
    cv2.imwrite(output_path, color_image)

# Example usage:
image_path = 'chess_images/test1.jpg'
split_and_predict_fen_with_overlay(image_path, output_path='chess_images/overlay_result2.jpg')


Generated FEN: wbwwwwwb/wwbwwwww/wwwwwwww/bwwbbbww/wwwbbbww/bwbwbwbb/bbbbbbbb/bwbwbwbw


In [3]:
# HOG and HSV combined Test

import joblib
import cv2
import numpy as np
from skimage.feature import hog

def extract_hog_features(image):
    image = cv2.resize(image, (60, 60))
    fd, _ = hog(image, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True)
    return fd

def extract_hsv_features(image, bins=16):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h_hist = cv2.calcHist([hsv_image], [0], None, [bins], [0, 180])
    s_hist = cv2.calcHist([hsv_image], [1], None, [bins], [0, 256])
    v_hist = cv2.calcHist([hsv_image], [2], None, [bins], [0, 256])
    h_hist = cv2.normalize(h_hist, h_hist).flatten()
    s_hist = cv2.normalize(s_hist, s_hist).flatten()
    v_hist = cv2.normalize(v_hist, v_hist).flatten()
    return np.concatenate((h_hist, s_hist, v_hist))

def process_chessboard(image_path, output_path='chess_images/overlay_result3.jpg'):
    hog_model = joblib.load('chess_images/prepared_data/tf_hog_random_forest_model.pkl')
    hsv_model = joblib.load('chess_images/prepared_data/bw_hsv_random_forest_model.pkl')
    
    image = cv2.imread(image_path)
    grayscale_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    color_image = image.copy()
    image = cv2.resize(image, (480, 480))
    grayscale_image = cv2.resize(grayscale_image, (480, 480))
    color_image = cv2.resize(color_image, (480, 480))
    
    step = 60
    predictions = []
    
    for row in range(8):
        row_data = []
        for col in range(8):
            square_gray = grayscale_image[row * step:(row + 1) * step, col * step:(col + 1) * step]
            square_color = image[row * step:(row + 1) * step, col * step:(col + 1) * step]
            
            # Check if square is occupied
            hog_features = extract_hog_features(square_gray)
            is_occupied = hog_model.predict([hog_features])[0]  # 1 = occupied, 0 = empty
            
            if is_occupied == 0:
                row_data.append("1")
                continue
            
            # Determine piece color
            hsv_features = extract_hsv_features(square_color)
            piece_color = hsv_model.predict([hsv_features])[0]  # 1 = white, -1 = black
            label = 'w' if piece_color == 1 else 'b'
            row_data.append(label)
            
            text_color = (255, 255, 255) if label == 'w' else (0, 0, 0)
            cv2.putText(color_image, label, (col * step + 20, row * step + 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
            
        # Convert row data to FEN format
        fen_row = ''.join(row_data)
        compact_fen_row = ''
        count = 0
        for char in fen_row:
            if char == '1':
                count += 1
            else:
                if count > 0:
                    compact_fen_row += str(count)
                    count = 0
                compact_fen_row += char
        if count > 0:
            compact_fen_row += str(count)
        
        predictions.append(compact_fen_row)
    
    fen_result = '/'.join(predictions)
    print("Generated FEN:", fen_result)
    
    # Save overlay image
    cv2.imwrite(output_path, color_image)
    return fen_result

# Example usage
image_path = 'chess_images/test1.jpg'
fen_string = process_chessboard(image_path)


Generated FEN: 2wwww2/ww4ww/wwwwwwww/3bbb2/3bbb2/b6b/bbbbbbbb/8
